In [1]:
from ultralytics import YOLO
import cv2
import os
from datetime import datetime
import pytz
import shutil
from rtsp_conf import cctvs
import time


def detect_tram(
    date,
    cctv_name,
    model_path,  # Replace with your actual model path
    confidence_threshold=0.25
):
    """
    Detect trams in images using the trained model
    """
    image_directory = 'production/' + cctv_name + '/' + date + '/captured_frames'
    processed_directory = 'production/' + cctv_name + '/' + date + '/processed_frames/'
    output_directory = 'production/' + cctv_name + '/' + date + '/output'

    # Create output directory if it doesn't exist
    if not os.path.exists(output_directory):
        os.makedirs(output_directory)

    # Create processed directory if it doesn't exist
    if not os.path.exists(processed_directory):
        os.makedirs(processed_directory)
    
    # Load the trained model
    model = YOLO(model_path)
    print(f"Model loaded from: {model_path}")
    
    # Get Singapore timezone
    sg_tz = pytz.timezone('Asia/Singapore')
    
    # Process each image
    for filename in sorted(os.listdir(image_directory)):
        if filename.endswith(('.jpg', '.png')):
            try:
                # Full path to image
                image_path = os.path.join(image_directory, filename)
                
                # Start
                start_time = time.perf_counter()

                # Read image
                image = cv2.imread(image_path)
                if image is None:
                    print(f"Could not read image: {filename}")
                    continue
                
                # Get timestamp from filename (assuming format "frame_YYYYMMDD_HHMMSS.jpg")
                timestamp_str = filename.split('frame_')[1].split('.')[0]
                timestamp = datetime.strptime(timestamp_str, "%Y%m%d_%H%M%S")
                timestamp = sg_tz.localize(timestamp)
                
                # Perform detection
                results = model.predict(
                    source=image,
                    conf=confidence_threshold,
                    verbose=False
                )
                
                # Process results
                if len(results[0].boxes) > 0:
                    # Get the image for drawing
                    annotated_frame = results[0].plot()
                    
                    # Add timestamp to image
                    cv2.putText(
                        annotated_frame,
                        timestamp.strftime("%Y-%m-%d %H:%M:%S"),
                        (10, 30),
                        cv2.FONT_HERSHEY_SIMPLEX,
                        1,
                        (0, 255, 0),
                        2
                    )
                    
                    # Save annotated image
                    output_path = os.path.join(output_directory, f"detected_{filename}")
                    cv2.imwrite(output_path, annotated_frame)
                    
                    # Get confidence scores
                    confidences = [float(box.conf) for box in results[0].boxes]
                    max_conf = max(confidences)

                    # Move processed image
                    shutil.move(image_path, processed_directory)
                    
                    end_time = time.perf_counter()
                    execution_time = end_time - start_time
                    print(f"Tram detected in {filename} with confidence: {max_conf:.2f}. Processing time: {execution_time} secs.")
                
            except Exception as e:
                print(f"Error processing {filename}: {str(e)}")
                continue

# if __name__ == "__main__":
    

In [2]:
# Get current Singapore time
date = (datetime.now(pytz.timezone('Asia/Singapore'))).strftime('%Y%m%d')

print("Available cctv_names:")
for cctv_name in cctvs:
    print(f"- {cctv_name}")

cctv_name = input('Enter cctv_name: ')
if cctv_name not in cctvs:
    print(f"Invalid cctv_name: {cctv_name}")
    exit()

# If screenshots data are not available, stop the process
source_directory = 'production/' + cctv_name + '/' + date + '/captured_frames'
if os.path.exists(source_directory) == False:
    print('No screenshots data available. Please run capture_rtsp.py first.')
    exit()

# Replace with your actual model path
model_path = "build_custom_model/runs/train/20241209_231950/weights/best.pt"  # Update this path if needed

# Run detection
detect_tram(
    date,
    cctv_name,
    model_path,
    confidence_threshold=0.5
)

Available cctv_names:
- ayana_gym
- rimba_gym
- segara_gym
- residences_gym
- ayana_tram_stop
- rimba_tram_stop
- wana_front_tram_stop
- wana_back_tram_stop
- segara_lobby
- segara_tram_stop
- kubu_tram_stop
- spa_tram_stop
- villa_tram_stop
Model loaded from: build_custom_model/runs/train/20241209_231950/weights/best.pt
Tram detected in frame_20241210_094326.png with confidence: 0.85. Processing time: 2.7350980000000007 secs.
Tram detected in frame_20241210_094336.png with confidence: 0.85. Processing time: 0.7097167999999989 secs.
Tram detected in frame_20241210_094346.png with confidence: 0.85. Processing time: 0.7319540000000018 secs.
Tram detected in frame_20241210_094356.png with confidence: 0.85. Processing time: 0.7321942999999997 secs.
Tram detected in frame_20241210_094406.png with confidence: 0.85. Processing time: 0.7351461999999991 secs.
Tram detected in frame_20241210_094416.png with confidence: 0.85. Processing time: 0.7093562999999996 secs.
Tram detected in frame_202412

KeyboardInterrupt: 